# Process for cleaning url lists from SQL database

The url lists for classification of staffing and IT-Consultants start out as queries from pocitac database.
The format is usually a multi column csv file from which we are interested in the url column.
The files are usually associated with a particular classification, for example manual-classified-staffing.
This sequence of steps takes files from raw sql dumps and outputs at the end csv tables with three columns.


| url       | content         | class |
|-----------|-----------------|-------|
| acme.com  | we are staffing |   0   |


These output files in turn are used by scikit learn to train the models.
The model itself is saved to disk and used by the EMR job during the url-classification runs monthly.

In [10]:
"""
Extract data from csv files which are provided from the SQL database.

File Input format is 
id,url,reason,origin,dbl_verified,product_hits,added_at
1,acriter-sa.com,tech_consult,manual_jodi,NULL,0,2016-09-09
2,adduri.in,tech_consult,manual_jodi,NULL,0,2016-09-09

File Output format is a txt file with a single column one url per line


inputs:
    source_name, Filename/path of file containing raw url lists from database.
    output_name, Filename to use for file which will contain only the urls
return:
    None
"""
source_name = 'url_lists/tech_consult_manual_suppress_on_url.csv'
output_name = 'url_lists/tech_consult_manual_urls.txt'


import pandas as pd

print "Reading input file and extracting the url vertical to save to file"
df = pd.read_csv(source_name)
df[['url']].to_csv(output_name, index=False) # index=False keeps it from writing out the row number...

print "Printing out the first rows of the output file for verification"
df_out = pd.read_csv(output_name)
df_out.head()

print "ALL DONE"
df_out.head()

Reading input file and extracting the url vertical to save to file
Printing out the first rows of the output file for verification
ALL DONE


,url
0,acriter-sa.com
1,adduri.in
2,advancedit.com.br
3,aequitas-affinity.de
4,allcommcr.com.pg


# Manual Step: Run the output list from previous cell through the url-classifier in text saving mode.

## Steps
1. At command line, set paths for saving text file by editing set_flag/save_file_on and set folder path.
1. Set the flag now by executing `. set_flag/save_file_on`
1. Now run the source file with `cat <source file> | python bin/urlChecker.py`
1. All the urls in the file should have a corresponding file saved in the output folder set.

In [2]:
"""
This script extracts readable content from a set of raw html files.  It takes as input a file with tuples.
Each tuple is a filename, containing just one column of urls, and it's classification.  
It's output will be a file similarly named, and will contain three columns...
url, visible_text, class

inputs:
    sets, Array of tupples. Each tuple is a filepath/class.  File contails urls to extract content from.
    html_content, Filepath to directory containing raw html files for extraction
output:
    file will be generated with 'data_for_classification' appended to source name.
return:
    None
"""

#INPUTS:
sets = [
     ("url_lists/tech_consult_manual_urls.txt", 0)
#     ("url_lists/test.txt", 0)
]
html_content = 'html_content_tech_consult'


from bs4 import BeautifulSoup
from bs4 import Comment
from bs4 import CData
import os
import re
import lxml
import bin.get_visible_text as gt


def disk_content_using_file_list(flist, source_path, binary):
    """Create a csv file with url, content, binary per row from given list of urls and content on disk
        flist: path of file to parse for names.
        source_path: directory with html content, where file names are same as url
        binary: what category should these be considered, 0/1
    """
    with open(flist, 'r') as source, open("data_for_classification_{}.tsv".format(str(binary)),'w') as target:
        for line in source:
            path_file = "{}/{}".format(source_path, line.strip())
            if os.path.exists(path_file):
                with open("{}/{}".format(source_path, line.strip()), 'r') as html_source:
                    html_text = html_source.read().decode('ascii', 'ignore')
                    visible_text = gt.get_visible_text(html_text)
                    if len(visible_text) >= 20:
                        target.write("{}\t{}\t{}\n".format(
                            line.strip(),
                            visible_text.encode('ascii','ignore'),
                            binary
                        ))
                    else:
                        print "{} low content {}".format(line.strip(), visible_text.encode('ascii', 'ignore'))
            else:
                print "{} not found".format(path_file)




for fpath, binary in sets:
    disk_content_using_file_list(fpath, html_content, binary)
print "ALL DONE"

html_content_tech_consult/url not found
capasystems.com low content  
html_content_tech_consult/daltron.com.pg not found
html_content_tech_consult/neelkanthinfonet.com not found
stabilityit.com low content  
winprovit.pt low content  winprovit 
locohire.com low content  
html_content_tech_consult/outsourzing.com not found
html_content_tech_consult/peopletalents.com not found
html_content_tech_consult/cadmigos.com not found
inavante.com low content  
html_content_tech_consult/intabyte.com not found
html_content_tech_consult/janatec.com not found
html_content_tech_consult/mekonsoft.com not found
html_content_tech_consult/nexstara.com not found
html_content_tech_consult/obieeservices.com not found
html_content_tech_consult/qualinfotech.com not found
html_content_tech_consult/softwaretechnologylabs.com not found
stgxinc.com low content  
synaptikgroup.com low content  
html_content_tech_consult/tekerp.com not found
html_content_tech_consult/vineva.com not found
www.atstechsolutions.com low

html_content_tech_consult/atgcorp.com not found
html_content_tech_consult/incedo.de not found
html_content_tech_consult/pentalog.com not found
html_content_tech_consult/pentalog.fr not found
html_content_tech_consult/vangent.com not found
html_content_tech_consult/winwire.com not found
html_content_tech_consult/timberhorn.com not found
dewsoftware.com low content  shares 
html_content_tech_consult/3ktechnologies.com not found
html_content_tech_consult/systemguru.com not found
html_content_tech_consult/technology-solution.com not found
infinitisystems.com low content  
html_content_tech_consult/dice.co.uk not found
infowarets.com low content  
protechnicals.com low content  
html_content_tech_consult/dataexel.com not found
cannonsp.com low content  cannonsp.com 
html_content_tech_consult/hp.taleo.net not found
html_content_tech_consult/gtcsystems.com not found
f1techsolutions.com low content  
html_content_tech_consult/sohosquaresolutions.com not found
html_content_tech_consult/sigmanet